In [1]:
import os
from tqdm import trange

from sklearn.datasets import fetch_openml
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import torch
import wandb
from pytorch_lightning import loggers

import utils

In [2]:
if not os.path.isdir("./data"):
    !mkdir data

filename = "./data/mnist.npz"
if os.path.isfile(filename):
    digits, labels, dist_mat = np.load(filename).values()
    N = len(digits)
else:
    N = 10000
    mnist = fetch_openml("mnist_784", version=1)

    idx = np.random.choice(np.arange(len(mnist["data"])), size=N, replace=False)
    digits, labels = mnist["data"].values[idx], mnist["target"].astype(int).values[idx]

    digits /= digits.max()

    dist_mat = np.zeros((N, N), dtype=float)
    for i in trange(N):
        for j in range(i):
            dist_mat[i, j] = dist_mat[j, i] = ((digits[i] - digits[j]) ** 2).sum() ** 0.5
            
    np.savez(filename, digits=digits, labels=labels, dist_mat=dist_mat)

In [3]:
metrics = {
    "bmds_train_acc": [],
    "bmds_eval_acc": [],
    "bmds_eval_acc_bayes": [],
    "pca_train_acc": [],
    "pca_eval_acc": [],
}

k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for idx_train, idx_eval in k_fold.split(torch.arange(N)):
    digits_train, digits_eval = digits[idx_train], digits[idx_eval]
    labels_train, labels_eval = labels[idx_train], labels[idx_eval]
    dist_mat_train, dist_mat_eval = dist_mat[idx_train[:, None], idx_train], dist_mat[idx_train[:, None], idx_eval]
    
    bmds = utils.SklearnBMDS(bmds_train_kwargs={"max_dim": 100, "threshold": 0.1})
    
    run = wandb.init(project="mnist bmds train")
    logger = loggers.WandbLogger()
    x_train, std_train = bmds.fit_transform(dist_mat_train, max_epochs=2, logger=logger, enable_progress_bar=False, enable_model_summary=False)
    run.finish()
    
    run = wandb.init(project="mnist bmds eval")
    logger = loggers.WandbLogger()
    x_eval, std_eval = bmds.transform(dist_mat_eval, logger=logger, enable_progress_bar=False, enable_model_summary=False)
    run.finish()
    
    clf = utils.SklearnMLPClassifier(labels_train.max() + 1)
    run = wandb.init(project="mnist bmds clf")
    logger = loggers.WandbLogger()
    pred_train = clf.fit_predict(x_train, labels_train, std_train, logger=logger, enable_progress_bar=False, enable_model_summary=False)
    pred_eval = clf.predict(x_eval)
    pred_eval_bayes = clf.predict(x_eval, std_eval)
    logger.log_hyperparams({
        "train_acc": accuracy_score(pred_train, labels_train),
        "eval_acc": accuracy_score(pred_eval, labels_eval),
        "eval_acc_bayes": accuracy_score(pred_eval_bayes, labels_eval),
    })
    run.finish()
    
    metrics["bmds_train_acc"].append(accuracy_score(pred_train, labels_train))
    metrics["bmds_eval_acc"].append(accuracy_score(pred_eval, labels_eval))
    metrics["bmds_eval_acc_bayes"].append(accuracy_score(pred_eval_bayes, labels_eval))
    
    
    pca = PCA(n_components=bmds.dim)
    pca_train = pca.fit_transform(digits_train)
    pca_eval = pca.transform(digits_eval)
    
    clf = utils.SklearnMLPClassifier(labels_train.max() + 1)
    run = wandb.init(project="mnist pca clf")
    logger = loggers.WandbLogger()
    pred_train = clf.fit_predict(pca_train, labels_train, logger=logger, enable_progress_bar=False, enable_model_summary=False)
    pred_eval = clf.predict(pca_eval)
    logger.log_hyperparams({
        "train_acc": accuracy_score(pred_train, labels_train),
        "eval_acc": accuracy_score(pred_eval, labels_eval),
    })
    run.finish()
    
    metrics["pca_train_acc"].append(accuracy_score(pred_train, labels_train))
    metrics["pca_eval_acc"].append(accuracy_score(pred_eval, labels_eval))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antonii-belyshev (ai-prentice). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Learning the optimal train embedding...


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


dim,██▇▇▆▆▅▅▅▄▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
frac,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▄▅▆▆▆▆▆▇▆▇▆▇▇▇▇▇▇▇█▇▇███
loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
reg,▆█▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
dim,14.0
epoch,1
frac,0.18283
loss,0.01669


...

dim,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
reg,▁███████████████████████████████████████
total_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
dim,14.0
epoch,499
loss,30.87544
reg,21.51643
total_loss,52.39188


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be 

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▄▃▂▃▂▂▂▂▁▂▂▂▂▂▂▃▂▂▁▂▂▂▁▂▁▂▃▂▂▁▂▂▂▂▂▂▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,999
loss,0.35517
trainer/global_step,99999


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be 

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▅▁▄▅▁▄▂▃▃▄▃▂▂▃▂▂▆▃▄▂▂▄▃▂▄▄▅▂▄▆▄▇▃▃▂▂▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,999
loss,0.08335
trainer/global_step,99999


In [18]:
pd.set_option("display.max_colwidth", 10)

In [19]:
metrics_df = pd.DataFrame(metrics)
metrics_df

,bmds_train_acc,bmds_eval_acc,bmds_eval_acc_bayes,pca_train_acc,pca_eval_acc
0,0.930000,0.9015,0.9035,0.956875,0.9040
1,0.926125,0.9160,0.9200,0.951625,0.9200
2,0.923375,0.9000,0.9010,0.957875,0.9170
3,0.940250,0.9085,0.9135,0.955375,0.9060
4,0.935875,0.9165,0.9165,0.956000,0.9135


In [7]:
mean_values = metrics_df.mean()
std_values = metrics_df.std()

summary_df = pd.DataFrame([mean_values, std_values], index=['Mean', 'Std'])
summary_df

,bmds_train_acc,bmds_eval_acc,bmds_eval_acc_bayes,pca_train_acc,pca_eval_acc
Mean,0.931125,0.90850,0.910900,0.955550,0.912100
Std,0.006931,0.00777,0.008272,0.002387,0.006914
